In [1]:
from selenium import webdriver
import csv
import pandas as pd
import re

In [2]:
driver = webdriver.Chrome('chromedriver.exe')

In [6]:
import time

data_list = []
year = 2017
Day = 1
Month = 1
Start = 2
want = '/html/body/div/div[3]/div[1]/div/div[{}]/div'.format(Start)

for Month in range(1, 13):
    #データ取得
    while Day < 32:
        url = 'https://mimorin2014.blog.fc2.com/blog-date-{0}{1}{2}.html'.format(str(year).zfill(4), str(Month).zfill(2), str(Day).zfill(2))
        driver.get(url)
        time.sleep(2.3)

        if len(driver.find_elements_by_xpath(want)) > 0:
            #タイトルが「（独立系を含む）デイリー合算ランキング」で始まるものを探す
            aim = '/html/body/div/div[3]/div[1]/div/div[{}]/h2/a'.format(Start)
            test = driver.find_element_by_xpath(aim)
            test = test.text
            Match = re.match(r'（独立系を含む）デイリー合算ランキング', test)
            try:
                while Match == None:
                    Start+=1
                    aim = '/html/body/div/div[3]/div[1]/div/div[{}]/h2/a'.format(Start)
                    test = driver.find_element_by_xpath(aim)
                    test = test.text
                    Match = re.match(r'（独立系を含む）デイリー合算ランキング', test)
            except:
                pass

            if Match:
                want = '/html/body/div/div[3]/div[1]/div/div[{}]/div'.format(Start)
                test = driver.find_element_by_xpath(want)
                kinema_data = test.text
                DataRange = kinema_data.split('\n')

                for a in DataRange:
                    data_list.append(a)

            Day+=1
            Start = 2
        else:
            Day+=1
    
    Day = 1
    Start = 2
    want = '/html/body/div/div[3]/div[1]/div/div[{}]/div'.format(Start)


#余計なものを取り除く
error = ['続きを読む', 
         '順位 販売数 座席数 回数 館数 先週比 映画（作品名）',
         '']
max = len(data_list)

for bag in error:
    a = 0
    try:
        while a < max:
            if data_list[a] == bag:
                del data_list[a]
            a += 1
    except IndexError:
        max = len(data_list)

#日付を取り出す
karenda = []
a = 0
try:
    while a < len(data_list):
        Match = re.match(r'（独立系を含む）デイリー上映25分前販売数合計ランキング', data_list[a])
        if Match:
            #
            data_list[a] = re.sub(r"（独立系を含む）デイリー上映25分前販売数合計ランキング", "", data_list[a])
            data_list[a] = re.sub(r"（\d+時中間集計）", "", data_list[a])
            data_list[a] = re.sub(r"：", "", data_list[a])
            for day in range(0, 25):
                karenda.append(data_list[a])

            del data_list[a]
        
        a += 1
except IndexError:
    pass

#空白を取り除く
error = ['']
max = len(data_list)

for bag in error:
    a = 0
    try:
        while a < max:
            if data_list[a] == bag:
                del data_list[a]
            a += 1
    except IndexError:
        max = len(data_list)

print('data_list', len(data_list))
print('karenda', len(karenda))

#出力
df = pd.DataFrame(zip(karenda, data_list))
time = df.iloc[:, 0]
df = df.iloc[:, 1].str.split('　', expand=True)
df = pd.concat([df, time], axis = 1)
df.columns = ['順位', '販売数', '座席数', '回数', '館数', '先週比', '映画（作品名）', '日付']
df = df.reindex(columns=['日付', '順位', '販売数', '座席数', '回数', '館数', '先週比', '映画（作品名）'])
df['順位'] = df['順位'].str.strip('*')
df['販売数'] = df['販売数'].str.strip('*')
df['座席数'] = df['座席数'].str.strip('*')
df['回数'] = df['回数'].str.strip('*')
df['館数'] = df['館数'].str.strip('*')
df['先週比'] = df['先週比'].str.strip('*')

df.to_csv('20220523_{0}kinema.csv'.format(year))

data_list 2125
karenda 2125


In [8]:
Reiwa_zero = pd.read_csv('20220523_2017kinema.csv')
Reiwa_one = pd.read_csv('20220523_2018_12kinema.csv') 
Reiwa_two = pd.read_csv('20220518_2019_2022_MarchFinish.csv')

df = pd.merge(Reiwa_zero, Reiwa_one, how='outer')
df = pd.merge(df, Reiwa_two, how='outer')

del df['Unnamed: 0']
df

,日付,順位,販売数,座席数,回数,館数,先週比,映画（作品名）
0,20171008,1,61072,337367,1203,254,NaN,アウトレイジ 最終章
1,20171008,2,42655,253772,1303,270,53.6%,亜人
2,20171008,3,33670,347382,1056,252,NaN,ナラタージュ
3,20171008,4,28207,140057,826,217,49.9%,僕のワンダフル・ライフ
4,20171008,5,22731,130534,818,277,50.2%,ナミヤ雑貨店の奇蹟
...,...,...,...,...,...,...,...,...
39945,20220331,21,1071,12135,99,163,57.0%,ウエスト・サイド・ストーリー
39946,20220331,22,1039,21191,169,120,90.0%,映画しまじろう しまじろうと キラキラおうこくの おうじさま
39947,20220331,23,861,13003,121,232,57.9%,ナイル殺人事件
39948,20220331,24,654,6093,52,20,NaN,オートクチュール


In [9]:
df.to_csv('20220518_2019_2022_MarchFinish.csv')